In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="#009e84"> ch9. 11장 Retrieval의 효율개선을 위한 전처리(markdown)_키워드사전활용 </font></b>

# 벡터DB : Chroma vs Pinecone

- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    - https://www.pinecone.io 에서 api key 생성 → .env에 추가(PINECONE_API_KEY 등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('data/소득세법_with markdown.docx')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=200)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

194

In [4]:
# embedding : upstage의 solar-embedding-1-large-passage

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()

embedding = UpstageEmbeddings(model='solar-embedding-1-large')

In [4]:
len(embedding.embed_query('소득세법'))

4096

In [2]:
document_list[46].page_content

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 ~ 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 ~ 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 ~ 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 ~ 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 ~ 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 ~ 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17

In [7]:
%%time

# pinecone vector database 저장
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

# 업로드시 시간이 오래걸려서 경고가 뜨는데 안보이려면 아나콘다 프롬프트 llm 환경에서 conda install -c conda-forge ipywidgets

# 데이터를 처음 업로드할 때
# index_name = 'tax-index-markdown'
# database = PineconeVectorStore.from_documents(documents=document_list,
#                                               embedding=embedding,
#                                               index_name=index_name)

# 업로드한 벡터DB를 가져올 때
database = PineconeVectorStore(embedding=embedding, # 질문을 임베딩하여 유사도 검색
                               index_name=index_name)

CPU times: total: 5.95 s
Wall time: 39.9 s


# 2. 답변 생성 전 retrieval 확인

In [8]:
query = '연봉이 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3) # 기본 k 값 : 4

In [9]:
retrieved_docs[1].page_content

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 ~ 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 ~ 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 ~ 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 ~ 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 ~ 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 ~ 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17

In [10]:
# retrieved_docs[0].page_content

retrieved_doc = '\n\n---\n\n'.join([doc.page_content for doc in retrieved_docs])

In [11]:
# 두칸 위에있는거랑 완벽하게 같음

retriever = database.as_retriever(search_kwargs={'k':3})
retrieved_docs = retriever.invoke(query)

In [12]:
retrieved_docs[1].page_content

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 ~ 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 ~ 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 ~ 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 ~ 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 ~ 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 ~ 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17

# 3. 답변 생성

In [23]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(model='gapt-4o-mini',
                                          messages=[{'role':'system', 'content':'당신은 최고의 한국 소득세법 전문가입니다'},
                                                    {'role':'user', 'content':f'''-[context]를 참고해서 사용자의 질문에 10줄 이내로 답변해주세요
                                                     -[context] : {retrieved_doc}
                                                     -질문 : {query}'''}],
                                          temperature=0.2)

In [28]:
print(response.choices[0].message.content)

연봉이 5천만원인 직장인의 소득세를 계산하기 위해, 먼저 종합소득 과세표준에 따른 세율을 적용합니다.

1. 5천만원의 경우, 과세표준은 다음과 같이 계산됩니다:
   - 1,400만원 이하: 1,400만원 × 6% = 84만원
   - 1,400만원 초과 5,000만원 이하: 84만원 + (5,000만원 - 1,400만원) × 15% = 84만원 + 525만원 = 609만원

2. 근로소득세액공제를 적용합니다:
   - 총급여액이 3천 300만원 초과 5천만원 이하이므로, 공제액은 74만원 - [(5,000만원 - 3,300만원) × 8/1000] = 74만원 - 13.6만원 = 60.4만원 (단, 66만원보다 적지 않으므로 66만원 적용).

3. 최종 소득세는 609만원 - 66만원 = 543만원입니다.

따라서, 연봉이 5천만원인 직장인의 소득세는 약 543만원입니다.


In [13]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv('UPSTAGE_API_KEY'),
                base_url="https://api.upstage.ai/v1")
response = client.chat.completions.create(model='solar-pro2',
                                          messages=[{'role':'system', 'content':'당신은 최고의 한국 소득세법 전문가입니다'},
                                                    {'role':'user', 'content':f'''-[context]를 참고해서 사용자의 질문에 5줄 이내로 답변해주세요
                                                     -[context] : {retrieved_doc}
                                                     -질문 : {query}'''}],
                                          temperature=0.2)

In [14]:
print(response.choices[0].message.content)

연봉 5천만원의 근로소득세액은 다음과 같이 계산됩니다:  

1. **근로소득세액공제 적용**:  
   - 3,300만원 초과 7,000만원 이하 구간에 해당하므로,  
   **공제액 = 74만원 - (5,000만원 - 3,300만원) × 8/1,000 = 62.6만원** (66만원 미만 시 66만원 적용).  

2. **종합소득산출세액 계산**:  
   - 과세표준 5,000만원 시 세율 적용 → **624만원 + (5,000만원 - 1,400만원) × 15% = 624만원 + 540만원 = 1,164만원**.  

3. **최종 세액**:  
   - 산출세액(1,164만원) - 근로소득세액공제(66만원) = **1,098만원**.  

※ 단, 추가 공제(의료비, 자녀 등) 및 지방세(10%)는 별도 적용 필요.  

(답변: 5줄 이내 요약)  

**연봉 5천만원 시 근로소득세액공제로 66만원 공제 후, 종합소득산출세액 1,164만원에서 1,098만원이 최종 세액입니다.** (지방세 제외)


In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano')

In [16]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면

from langchain_upstage import ChatUpstage

llm = ChatUpstage(model = 'solar-pro2',
                  reasoning_effort='high') # 느리지만 더 깊게 추론함 (low, medium)

In [17]:
prompt = f'''[identity]
-당신은 최고의 한국 소득세법 전문가입니다
-[context]를 참고해서 사용자의 질문에 답변해주세요
-[context]는 다음과 같습니다
{retrieved_doc}
-질문 : {query}'''

In [18]:
ai_message = llm.invoke(prompt)

In [19]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산하는 과정은 다음과 같습니다:

---

### 1. **종합소득산출세액 계산 (제55조 세율 적용)**
- **총급여액**: 5,000만원  
- **과세표준**: 문제에서는 공제 항목을 명시하지 않았으므로, 총급여액을 과세표준으로 가정합니다.  
- **세율 적용**:  
  - 1,400만원 이하: 6%  
  - 1,400만원 초과 ~ 5,000만원 이하: **84만원 + (1,400만원 초과금액의 15%)**  
  - 계산:  
    ```
    84만원 + (5,000만원 - 1,400만원) × 15%  
    = 84만원 + 3,600만원 × 15%  
    = 84만원 + 540만원 = **624만원**
    ```

---

### 2. **근로소득세액공제 적용 (제59조)**
- **총급여액**: 5,000만원 (3,300만원 초과 7,000만원 이하 구간)  
- **공제액 계산**:  
  ```
  74만원 - [(5,000만원 - 3,300만원) × 8/1,000]  
  = 74만원 - (1,700만원 × 0.008)  
  = 74만원 - 13.6만원 = **60.4만원**
  ```  
- **최소 공제액 조건**: 계산 결과가 66만원 미만이므로 **66만원** 적용.

---

### 3. **최종 소득세 산정**
- **종합소득산출세액**: 624만원  
- **공제 후 세액**:  
  ```
  624만원 - 66만원 = **558만원**
  ```

---

### 4. **추가 고려사항**
- **자녀세액공제**: 질문자에 자녀 정보가 없으므로 제외합니다.  
- **일용근로자 공제**: 일반 직장인은 해당되지 않습니다.  
- **지방세**: 소득세의 10% (55.8만원)가 추가되나, 질문 범위 외입니다.

---

### **최종 답변**
연봉 5천만원인 직장인의 **소득세는 558만원**입니다.  

> 📌 **참고**: 실제 계산에서는 보험료, 의료비 등 추가 

# 4. langchain으로 답변 생성

In [25]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage
from dotenv import load_dotenv
load_dotenv()

# 1. LLM과 임베딩 초기화
# llm = ChatOpenAI(model='gpt-4.1-mini')
llm = ChatUpstage(model='solar-pro2')
embedding = UpstageEmbeddings(model='solar-embedding-1-large')

# 2. vertor store load
index_name = 'tax-index-markdown'
vectorstore = PineconeVectorStore(embedding=embedding, 
                               index_name=index_name)

# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type='similarity',
                                     search_kwargs={'k':4})

# 4. 프롬프트 템플릿
template = f'당신은 최고의 한국 소득세 전문가입니다. 다음 문맥을 참고하여 질문에 답하세요. 답을 모르면 모른다고 답하세요. 최대 3문장으로 간결하게 답변하세요. 질문 : {{query}} / 문맥 : {{context}} / 답변 :'
prompt = ChatPromptTemplate.from_template(template)

# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])

In [26]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {'query':'~'} → '~'
rag_chain = ({'context':retriever | format_documents, 'query':RunnablePassthrough()} # 질문 그대로 전달
             | prompt # prompt에 context와 query 변수 주입
             | llm    # llm에 prompt invoke
             | StrOutputParser()) 

# 7. 실행
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
rag_chain.invoke(query)

'연봉 5천만원의 근로소득 세액공제 금액은 **66만원**입니다.  \n(근로소득세액공제 규정 제59조②2항 적용: 총급여액 3,300만원 초과 7천만원 이하 시, 계산식 적용 결과 66만원 미만이 아니므로 66만원 유지)  \n\n※ 단, 종합소득산출세액(제55조), 자녀세액공제(제59조의2) 등 추가 공제요소 반영 시 실제 납부세액은 달라질 수 있습니다.'

# 5. 키워드 사전 활용

In [31]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)

embedding = UpstageEmbeddings(model='solar-embedding-1-large')

In [33]:
vec1 = embedding.embed_query('총급여')
vec2 = embedding.embed_query('연봉')
print('총급여와 연봉의 유사도 :', cosine_similarity(vec1, vec2))

총급여와 연봉의 유사도 : 0.6874992849327983


In [38]:
vec1 = embedding.embed_query('종합소득')
vec2 = embedding.embed_query('연봉')
print('종합소득과 연봉의 유사도 :', cosine_similarity(vec1, vec2))

종합소득과 연봉의 유사도 : 0.5662298146748994


In [36]:
vec1 = embedding.embed_query('직장인')
vec2 = embedding.embed_query('거주자')
print('직장인과 거주자의 유사도 :', cosine_similarity(vec1, vec2))

직장인과 거주자의 유사도 : 0.540122403716264


In [37]:
vec1 = embedding.embed_query('5천만원')
vec2 = embedding.embed_query('5,000만원')
print('5천만원과 5,000만원의 유사도 :', cosine_similarity(vec1, vec2))

5천만원과 5,000만원의 유사도 : 0.9122554959328768


In [40]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = vectorstore.similarity_search(query, k=2)
retrieved_docs[1].page_content

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 ~ 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 ~ 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 ~ 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 ~ 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 ~ 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 ~ 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17

In [42]:
# 거주자와 직장인의 유사도가 높지 않아서 지금까지 55조를 1번째 방에 가져왔던것. 그래서 질문하지전에 단어를 한번 바꾸는 작업 필요함

query = '총급여가 5000만원인 거주자의 소득세는 얼마인가요?'
retrieved_docs = vectorstore.similarity_search(query, k=2)
retrieved_docs[0].page_content

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6퍼센트 |\n\n| 1,400만원 초과 ~ 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트) |\n\n| 5,000만원 초과 ~ 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 ~ 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트) |\n\n| 1억5천만원 초과 ~ 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트) |\n\n| 3억원 초과 ~ 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트) |\n\n| 5억원 초과 ~ 10억원 이하 | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트) |\n\n| 10억원 초과 | 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트) |\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17

In [64]:
# 사람을 나타내는 표현 → 거주자 / 5천만원 → 5000만원 

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model='gpt-4o-mini')
# llm = ChatUpstage(model='solar-pro2')

dictionary = ['사람을 나타내는 표현 -> 거주자']
prompt = ChatPromptTemplate.from_template(f'''사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요
만약 변경할 필요가 없을 경우, 사용자의 질문을 변경하지 않아도 됩니다. 질문만 리턴해주세요
사전 : {dictionary}
질문 : {{question}}''')

In [54]:
parser = StrOutputParser()
parser.invoke(llm.invoke(prompt.invoke({'question':'소득이 높은 남자가 있습니다'})))

'소득이 높은 거주자가 있습니다.'

In [65]:
dictionary_chain = prompt | llm | StrOutputParser()
dictionary_chain.invoke('소득이 높은 남자가 5천만원을 가지고 있습니다')

'소득이 높은 거주자가 5천만원을 가지고 있습니다.'

In [66]:
dictionary_chain.invoke('연봉 5천만원인 직장인의 소득세는 얼마에요?')

'연봉 5천만원인 거주자의 소득세는 얼마에요?'

In [67]:
# rag_chain.invoke('연봉 5천만원인 직장인의 소득세는 얼마에요?')

final_chain = dictionary_chain | rag_chain

In [68]:
final_chain.invoke('연봉 5천만원인 직장인의 소득세는 얼마에요?')

'제공된 문맥은 근로소득세액 공제 기준을 설명하고 있으며, **연봉 5천만원**의 경우 **총급여액이 3천300만원 초과 7천만원 이하**에 해당합니다.  \n\n계산에 따라:  \n- **74만원 - [(5,000만원 - 3,300만원) × 8/1,000] = 74만원 - 1,280만원 × 0.008 = 74만원 - 10.24만원 = 63.76만원**  \n- 단, **66만원 미만이므로 66만원**이 적용됩니다.  \n\n따라서 **근로소득세액 공제 전 산출세액은 66만원**입니다. 단, 실제 소득세는 종합소득과세표준(공제 후 금액)과 세율표(제55조)를 적용해야 하며, 추가 공제(자녀세액공제 등)가 있을 수 있습니다.  \n\n간결한 답변:  \n**근로소득세액 공제액은 66만원입니다.** (단, 실제 소득세는 종합소득산출세액 및 공제 적용 필요)'

In [ ]:
# 입력 {'question':'질문'} → dictionary_chain이 질문 멘트를 개선 → 개선된 질문을 rag_chain에 전달 
# → retriever가 관련된 문서를 검색 → format_documents함수 → 완성된 prompt llm에 전달하여 답변 생성 → 문자만 추출해서 최종답변